In [ ]:
from chatbot import Chatbot

api_url = "http://localhost:11434"

In [ ]:
cow_bot = Chatbot(
            system_prompt="You are a cow, moo!",
            api_url=api_url
        )

print(cow_bot.generate_response("Hello!"))

In [ ]:
print(cow_bot.generate_response("hello, how do i install python on windows?"))

In [ ]:
install_python_windows_response = cow_bot.generate_response("hello install python windows")

print(install_python_windows_response)

In [ ]:
print(cow_bot.generate_response("What is the etymology of the word hello?"))

In [ ]:
import json

category_bot = Chatbot(
            system_prompt="""
                You are tasked with categorizing text. 
                
                Respond as a JSON object with the following keys: 
                - 'cow': bool indicating if the response is presented by a cow,
                - 'subject': string indicating the subject of the response
            """,
            api_url=api_url,
            format="json"
        )

category_bot_response = category_bot.generate_response(install_python_windows_response)

print(category_bot_response)

response_as_json = json.loads(category_bot_response)

if response_as_json["cow"]:
    print("Subject: ", response_as_json["subject"])